<img src="https://raw.githubusercontent.com/maverick-faus/Files/master/DeepDeamon2.png" alt="drawing" width="100px"/>
# Red 4B - Testing
TT: Deep Deep Dishes  
Food recommendation System  
ESCOM  
By Faus - 2K18

In [0]:
import tensorflow as tf
import numpy as np
import pickle 
from sklearn.metrics import confusion_matrix

In [22]:
!pip install PyDrive
import os
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
fileIdLabelmap = '1WFD03892EOtRIfEYKOAJRilqiEGzzDTX'
#label map
downloaded1 = drive.CreateFile({'id': fileIdLabelmap})
downloaded1.GetContentFile("4B3.ckpt.data-00000-of-00001")

fileIdLabelmap = '1vJMT8uw1aXL0p62wzOpEuCCZp6P_HsNQ'
#label map
downloaded1 = drive.CreateFile({'id': fileIdLabelmap})
downloaded1.GetContentFile("4B3.ckpt.index")

fileIdLabelmap = '1c7JNawj9Tygm4c6eNC2mkgzAJmjTbm5d'
#label map
downloaded1 = drive.CreateFile({'id': fileIdLabelmap})
downloaded1.GetContentFile("4B3.ckpt.meta")

fileIdLabelmap = '1fPNItsTY_q74Wxq2V3oPgTlig4d1S8-Z'
#label map
downloaded1 = drive.CreateFile({'id': fileIdLabelmap})
downloaded1.GetContentFile("4B_test.pickle")

In [26]:
!ls

4B3.ckpt.data-00000-of-00001  4B3.ckpt.meta   adc.json	drive
4B3.ckpt.index		      4B_test.pickle  CNNClass	sample_data


In [0]:
def deep_neural_convolutional_class(
    batch_size=20,
    image_size=[200,200],
    Drop_prob=1.0,
    learning_rate = 1e-3,
    n_nodes_hl0 = 2000,
    n_nodes_hl1 = 1000,
    n_nodes_hl2 = 500,
    n_nodes_hl3 = 100,
    n_classes=17
    ):
    
    reset_graph()
    #Place holder de entrada 
    x= tf.placeholder(tf.float32,[batch_size,image_size[0],image_size[1],3], name='placeholder_img_entrada')
    y=tf.placeholder(tf.float32,name='placeholder_one_hot')
  
    #Diccionario de pesos convolucionales 
    with tf.name_scope('pesos_bias') as scope1:
        weigths={"w_conv1":tf.Variable(tf.random_normal([5,5,3,32]),name='Pesos_1_32'),
                 "w_conv2":tf.Variable(tf.random_normal([5,5,32,64]),name='Pesos_1_64'),
                 "w_conv3":tf.Variable(tf.random_normal([3,3,64,128]),name='Pesos_1_128'),   
                 "w_conv4":tf.Variable(tf.random_normal([5,5,128,256]),name='Pesos_1_256'),
                 "w_conv5":tf.Variable(tf.random_normal([5,5,256,512]),name='Pesos_1_512'),
                 "w_conv6":tf.Variable(tf.random_normal([5,5,512,1024]),name='Pesos_1_1024'),
                }
        #Diccionario de bias
        biases={"b_conv1":tf.Variable(tf.random_normal([32]),name='Bias_1_32'),
                "b_conv2":tf.Variable(tf.random_normal([64]),name='Bias_1_64'),
                "b_conv3":tf.Variable(tf.random_normal([128]),name='Bias_1_128'),
                "b_conv4":tf.Variable(tf.random_normal([256]),name='Bias_1_256'),
                "b_conv5":tf.Variable(tf.random_normal([512]),name='Bias_1_512'),
                "b_conv6":tf.Variable(tf.random_normal([1024]),name='Bias_1_1024'),
               }

    #Extractor de características
    with tf.name_scope('capas_conv') as scope2:
        conv1=tf.nn.relu(conv2d(x,weigths["w_conv1"],'Capa_Conv_1','SAME')+biases["b_conv1"],name='Func_relu_1')
        conv1=tf.nn.dropout(conv1,Drop_prob)
        conv1=maxpool2d(conv1,ks=[1,2,2,1],st=[1,2,2,1])
        #imagen resultante de 100x100x32
        print(conv1)

        conv2=tf.nn.relu(conv2d(conv1,weigths["w_conv2"],'Capa_Conv_2','SAME')+biases["b_conv2"],name='Func_relu_2')
        conv2=tf.nn.dropout(conv2,Drop_prob)
        conv2=maxpool2d(conv2,ks=[1,2,2,1],st=[1,2,2,1])
        #imagen resultante de 50x50x64
        print(conv2)

        conv3=tf.nn.relu(conv2d(conv2,weigths["w_conv3"],'Capa_Conv_3','VALID')+biases["b_conv3"],name='Func_relu_3')
        conv3=tf.nn.dropout(conv3,Drop_prob)
        conv3=maxpool2d(conv3,ks=[1,2,2,1],st=[1,2,2,1])
        #imagen resultante de 24x24x128
        print(conv3)

        conv4=tf.nn.relu(conv2d(conv3,weigths["w_conv4"],'Capa_Conv_4','SAME')+biases["b_conv4"],name='Func_relu_4')
        conv4=tf.nn.dropout(conv4,Drop_prob)
        conv4=maxpool2d(conv4,ks=[1,2,2,1],st=[1,2,2,1])
        #imagen resultante de 12x12x256
        print(conv4)

        conv5=tf.nn.relu(conv2d(conv4, weigths["w_conv5"],'Capa_Conv_5','SAME')+biases["b_conv5"],name='Func_relu_5')
        conv5=tf.nn.dropout(conv5,Drop_prob)
        conv5=maxpool2d(conv5,ks=[1,2,2,1],st=[1,2,2,1])
        #vector para clasificar de 6x6x512
        print(conv5)
        
        #Embeding, son las caracteristicas fonales que se pasarán al MLP o red completamente conectada para clasifiacar
        embdeding=tf.reshape(conv5,[batch_size,6*6*512],name='Embeding')
        print(embdeding)
    
    #Red perceptron, declaración de capas, son diccionarios de pesos y bias.
    with tf.name_scope('capas_clasificador') as scope3:
        hidden_0_layer = {'weights':tf.Variable(tf.random_normal([6*6*512, n_nodes_hl0]),name='Capa_oculta_pesos_0'),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl0]),name='Capa_oculta_bias_0')}

        hidden_1_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl0, n_nodes_hl1]),'Capa_oculta_pesos_1'),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl1]),name='Capa_oculta_bias_1')}

        hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2]),'Capa_oculta_pesos_2'),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl2]),name='Capa_oculta_bias_2')}
        
        hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3]),'Capa_oculta_pesos_3'),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl3]),name='Capa_oculta_bias_3')}

        output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes]),'Capa_salida_pesos'),
                        'biases':tf.Variable(tf.random_normal([n_classes]),name='Capa_salida_bias'),}
    
    #W*P + B 
    with tf.name_scope('op_clasificador') as scope4:
        
        l0 = tf.add(tf.matmul(embdeding,hidden_0_layer['weights'],name='Matmul_l0'), hidden_0_layer['biases'],name='Suma_Pesos_Bias_0')
        l0 = tf.nn.relu(l0,name='l0_relu_0')

        l1 = tf.add(tf.matmul(l0,hidden_1_layer['weights'],name='Matmul_l1'), hidden_1_layer['biases'],name='Suma_Pesos_Bias_1')
        l1 = tf.nn.relu(l1,name='l1_relu_1')

        l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights'],name='Matmul_l2'), hidden_2_layer['biases'],name='Suma_Pesos_Bias_2')
        l2 = tf.nn.relu(l2,name='l2_relu_2')
        
        l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights'],name='Matmul_l3'), hidden_3_layer['biases'],name='Suma_Pesos_Bias_3')
        l3 = tf.nn.relu(l3,name='l3_relu_3')

        output = tf.matmul(l3,output_layer['weights'],name='Matmul_out') + output_layer['biases']
    
    
    
    
    
    # Declarando la funcion de costo y entrenamiento
    #Reduce mean, reduce la dimencion del tensor en un promedio es decir hace el promedio del costo o error
    y1,y2,y3,y4= tf.split(y,[5,5,4,3],1) #PaRTIR EL VECTOR EN CADA ONE HOT 
    print(y1)
    print(y2)
    print(y3)
    print(y4)
    print("----")
    output1,output2,output3,output4= tf.split(output, [5,5,4,3], 1)
    print(output)
    print(output1)
    print(output2)
    print(output3)
    print(output4)
    
    
    with tf.name_scope('costo_y_optimizador') as scope5:
        cost1 = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=output1,labels=y1,name='softmax_cross_entropy_with_logits1'),name='reduce_mean1')
        cost2 = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=output2,labels=y2,name='softmax_cross_entropy_with_logits2'),name='reduce_mean2')
        cost3 = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=output3,labels=y3,name='softmax_cross_entropy_with_logits3'),name='reduce_mean3')
        cost4 = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=output4,labels=y4,name='softmax_cross_entropy_with_logits4'),name='reduce_mean4')
        cost = cost1+cost2+cost3+cost4
        optimizer = tf.train.AdamOptimizer(name='Adam',learning_rate=learning_rate).minimize(cost,name='minimo')
    
    
    tf.summary.scalar("costo",cost)
    
    
    #correct = tf.equal(tf.argmax(output,1),tf.argmax(y,1),name='valores_correctos')
    #accuracy = tf.reduce_mean(tf.cast(correct,'float'),name='Promedio_exactitud') #porcentaje de error

    
    correct_1 = tf.equal(tf.argmax(output1,1),tf.argmax(y1,1),name='valores_correctos1')
    accuracy_1 = tf.reduce_mean(tf.cast(correct_1,'float'),name='Promedio_exactitud1') #porcentaje de error
    
    correct_2 = tf.equal(tf.argmax(output2,1),tf.argmax(y2,1),name='valores_correctos2')
    accuracy_2 = tf.reduce_mean(tf.cast(correct_2,'float'),name='Promedio_exactitud2') #porcentaje de error
    
    correct_3 = tf.equal(tf.argmax(output3,1),tf.argmax(y3,1),name='valores_correctos3')
    accuracy_3 = tf.reduce_mean(tf.cast(correct_3,'float'),name='Promedio_exactitud3') #porcentaje de error
    
    correct_4 = tf.equal(tf.argmax(output4,1),tf.argmax(y4,1),name='valores_correctos4')
    accuracy_4 = tf.reduce_mean(tf.cast(correct_4,'float'),name='Promedio_exactitud4') #porcentaje de error

    
    avg=accuracy_1*0.3+accuracy_2*0.3+accuracy_3*0.2+accuracy_4*0.2
    
    
    
    
    tf.summary.scalar("average",avg)
    summaries = tf.summary.merge_all()
    
    return dict(
        x = x,
        y=y,
        embeding=conv5,
        output=output,
        saver = tf.train.Saver(),
        total_loss = cost,
        train_step = optimizer,
        summaries = summaries,
        average = avg,
        accuracy_precio = accuracy_1,
        accuracy_estilo = accuracy_2,
        accuracy_pan = accuracy_3,
        accuracy_side = accuracy_4,
        loss_precio = cost1,
        loss_estilo = cost2,
        loss_pan = cost3,
        loss_side = cost4
    )

In [0]:
def conv2d(x, W,name,padd,strid=[1,1,1,1]):
    #El stride de esa función no reduce el tamaño de la imagen
    return tf.nn.conv2d(x, W, strides=strid, padding=padd,name=name)

def maxpool2d(x,ks,st):
    #           El st de esta función reduce la imagen a la mitad
    return tf.nn.max_pool(x, ksize=ks, strides=st, padding='SAME')

In [0]:
def reset_graph():
    #Limpiamos la gráfic
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

In [0]:
def check_net(g, test, checkpoint):
   with tf.Session() as sess:
        writer = tf.summary.FileWriter("./CNNClass")
        tf.summary.FileWriter.add_graph(writer,sess.graph)
        #sess.run(tf.initialize_all_variables())
        g['saver'].restore(sess, checkpoint)
        new_Y=[]
        for j in range(test.shape[0]): #Unir cada etiqueta que viene por separado a un vector de 17 posiciones
            epoch_Y1=test[j,1]
            epoch_Y2=test[j,2]
            epoch_Y3=test[j,3]
            epoch_Y4=test[j,4]
            new_Y.append(epoch_Y1+epoch_Y2+epoch_Y3+epoch_Y4)
        
        
        #print(new_Y[:])
        
        feed_dict={g['x']: test[:,0].tolist(),
                   g['y']: new_Y[:]}
        preds,avg = sess.run([g['output'],g['average']], feed_dict)
        return preds,avg


In [32]:
path= '4B_test'
data1=np.array(pickle.load(open(path+".pickle","rb"))) #np.load()
print(data1.shape)
g=deep_neural_convolutional_class(batch_size = 200)#Batch de prueba
pred,avg=check_net(g,data1,"./4B3.ckpt")

(200, 5)
Tensor("capas_conv/MaxPool:0", shape=(200, 100, 100, 32), dtype=float32)
Tensor("capas_conv/MaxPool_1:0", shape=(200, 50, 50, 64), dtype=float32)
Tensor("capas_conv/MaxPool_2:0", shape=(200, 24, 24, 128), dtype=float32)
Tensor("capas_conv/MaxPool_3:0", shape=(200, 12, 12, 256), dtype=float32)
Tensor("capas_conv/MaxPool_4:0", shape=(200, 6, 6, 512), dtype=float32)
Tensor("capas_conv/Embeding:0", shape=(200, 18432), dtype=float32)
Tensor("split:0", dtype=float32)
Tensor("split:1", dtype=float32)
Tensor("split:2", dtype=float32)
Tensor("split:3", dtype=float32)
----
Tensor("op_clasificador/add:0", shape=(200, 17), dtype=float32)
Tensor("split_1:0", shape=(200, 5), dtype=float32)
Tensor("split_1:1", shape=(200, 5), dtype=float32)
Tensor("split_1:2", shape=(200, 4), dtype=float32)
Tensor("split_1:3", shape=(200, 3), dtype=float32)
INFO:tensorflow:Restoring parameters from ./4B3.ckpt


In [33]:
print(avg)

0.3265


In [0]:
pred_precio=[]
pred_estilo=[]
pred_pan=[]
pred_guar=[]

for elem in pred:

  p1,p2,p3,p4,p5= np.split(elem, [5,10,14,18])
  pred_precio.append(p1)
  pred_estilo.append(p2)
  pred_pan.append(p3)
  pred_guar.append(p4)
  
precio=np.asarray(pred_precio)
estilo=np.asarray(pred_estilo)
pan=np.asarray(pred_pan)
guar=np.asarray(pred_guar)

**Testing Precio**

In [67]:
lista = []
for item in precio:
    lista.append(list(item).index(max(item)))
print(lista)

[1, 1, 2, 2, 3, 2, 4, 3, 1, 3, 3, 1, 3, 3, 1, 1, 3, 1, 1, 1, 3, 4, 1, 1, 1, 3, 3, 1, 1, 4, 1, 1, 3, 1, 3, 1, 0, 1, 0, 1, 1, 1, 2, 1, 3, 1, 1, 1, 1, 3, 1, 3, 3, 1, 1, 1, 1, 0, 1, 1, 3, 3, 3, 3, 4, 2, 3, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 3, 1, 1, 3, 1, 1, 4, 3, 1, 1, 0, 1, 4, 1, 1, 4, 3, 3, 1, 3, 1, 1, 3, 1, 1, 1, 0, 3, 1, 2, 1, 1, 1, 3, 3, 1, 1, 1, 2, 1, 1, 1, 3, 3, 1, 1, 3, 3, 1, 1, 3, 1, 1, 1, 4, 0, 1, 1, 1, 3, 3, 1, 3, 1, 3, 0, 3, 4, 3, 1, 3, 3, 4, 1, 1, 1, 4, 3, 3, 3, 1, 3, 3, 4, 1, 1, 1, 1, 3, 1, 3, 1, 4, 1, 1, 0, 1, 3, 1, 1, 1, 1, 1, 3, 0, 0, 0, 1, 3, 0, 3, 1, 3, 2, 3, 1, 1, 3, 3]


In [68]:
l = []
for i in range(200):
    l.append(data1[i][1].index(1))
print(l)

[1, 3, 3, 0, 1, 3, 0, 2, 2, 0, 1, 2, 2, 4, 2, 2, 2, 2, 3, 3, 1, 0, 2, 2, 1, 1, 0, 1, 0, 1, 1, 2, 0, 1, 3, 2, 1, 2, 1, 0, 3, 2, 1, 2, 4, 1, 0, 1, 0, 0, 1, 2, 0, 1, 1, 3, 3, 1, 1, 3, 2, 3, 4, 1, 2, 2, 0, 1, 0, 2, 0, 2, 1, 3, 1, 4, 1, 0, 1, 3, 2, 1, 0, 0, 2, 1, 2, 0, 1, 0, 1, 3, 2, 1, 1, 2, 3, 1, 1, 3, 4, 4, 0, 2, 2, 1, 3, 3, 1, 2, 1, 1, 3, 0, 3, 3, 1, 2, 4, 1, 2, 1, 1, 2, 1, 1, 1, 2, 2, 2, 4, 1, 2, 1, 0, 0, 2, 2, 3, 0, 2, 2, 1, 1, 4, 2, 1, 2, 4, 1, 3, 3, 1, 3, 1, 0, 2, 3, 2, 4, 1, 1, 1, 3, 3, 2, 2, 0, 2, 0, 3, 4, 1, 1, 1, 2, 1, 1, 1, 0, 3, 3, 2, 2, 4, 2, 2, 1, 1, 3, 0, 4, 2, 2, 4, 2, 1, 3, 1, 3]


In [69]:
y_test = lista
y_pred = l
matriz_conf = confusion_matrix(y_true = y_test,y_pred=y_pred)
print(matriz_conf)

[[ 1  6  3  2  0]
 [17 32 33 18  5]
 [ 1  6  1  2  1]
 [ 7 19 17  8  8]
 [ 4  3  1  4  1]]


**Testing Estilo**

In [70]:
lista = []
for item in estilo:
    lista.append(list(item).index(max(item)))
print(lista)

[1, 4, 4, 1, 1, 3, 1, 1, 0, 1, 0, 1, 1, 3, 4, 4, 1, 1, 4, 4, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 3, 1, 1, 1, 1, 1, 0, 1, 1, 0, 4, 1, 4, 2, 1, 3, 4, 1, 0, 1, 1, 1, 0, 1, 1, 0, 2, 1, 2, 1, 4, 4, 0, 3, 1, 1, 3, 0, 2, 1, 2, 1, 4, 2, 1, 0, 1, 1, 2, 3, 1, 1, 1, 1, 2, 0, 1, 3, 1, 1, 1, 1, 1, 2, 0, 3, 1, 1, 4, 1, 1, 1, 4, 1, 3, 0, 2, 3, 0, 0, 1, 1, 4, 1, 2, 2, 1, 0, 1, 0, 4, 3, 1, 1, 1, 0, 1, 1, 4, 1, 1, 4, 1, 2, 1, 1, 1, 4, 4, 0, 1, 4, 4, 0, 1, 4, 2, 4, 1, 4, 1, 4, 1, 1, 2, 4, 4, 4, 1, 0, 1, 4, 2, 4, 4, 1, 1, 3, 4, 4, 1, 0, 1, 1, 1, 1, 3, 0, 1, 3, 2, 1, 4, 0, 0, 1, 1, 3, 1, 1, 1, 1, 2, 1, 3, 1, 3, 1]


In [71]:
l = []
for i in range(200):
    l.append(data1[i][2].index(1))
print(l)

[2, 3, 4, 1, 3, 3, 1, 3, 3, 0, 2, 3, 3, 4, 1, 4, 4, 3, 3, 3, 2, 1, 3, 3, 3, 2, 0, 0, 2, 3, 1, 3, 0, 2, 3, 2, 2, 3, 3, 0, 4, 1, 1, 2, 3, 1, 2, 1, 3, 1, 1, 2, 1, 1, 0, 4, 3, 2, 1, 4, 2, 4, 4, 2, 1, 4, 1, 1, 0, 3, 2, 3, 1, 4, 2, 4, 1, 0, 3, 3, 2, 2, 1, 1, 0, 2, 4, 0, 1, 3, 1, 2, 3, 2, 2, 2, 4, 2, 1, 4, 4, 4, 0, 3, 3, 2, 1, 4, 3, 3, 3, 3, 4, 0, 3, 3, 3, 3, 4, 3, 3, 2, 2, 3, 1, 0, 3, 3, 3, 3, 3, 1, 4, 1, 0, 0, 3, 4, 3, 1, 1, 3, 3, 3, 4, 3, 3, 3, 4, 3, 3, 4, 1, 3, 3, 1, 3, 4, 2, 4, 3, 3, 3, 4, 4, 3, 0, 1, 2, 1, 2, 4, 2, 2, 2, 1, 2, 1, 0, 0, 3, 3, 4, 3, 4, 1, 3, 1, 1, 3, 2, 4, 4, 2, 4, 1, 0, 3, 1, 4]


In [72]:
y_test = lista
y_pred = l
matriz_conf = confusion_matrix(y_true = y_test,y_pred=y_pred)
print(matriz_conf)

[[ 4  4  5 12  2]
 [ 9 25 20 33 15]
 [ 1  3  4  4  6]
 [ 4  4  2  6  2]
 [ 1  5  5 13 11]]


**Testing Pan**

In [73]:
lista = []
for item in pan:
    lista.append(list(item).index(max(item)))
print(lista)

[0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 2, 0, 0, 1, 0, 0, 2, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 2, 0, 0, 1, 1, 0, 2, 0, 1, 2, 0, 2, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 2, 1, 1, 0, 2, 1, 1, 1, 0, 0, 0, 2, 3, 1, 0, 2, 0, 0, 0, 0, 1, 0, 2, 0, 2, 0, 0, 1, 2, 0, 0, 1, 0, 1, 1, 0, 1, 0, 3, 1, 0, 0, 1, 2, 1, 0, 0, 1, 0, 1, 1, 3, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 2, 2, 0, 2, 0, 2, 0, 1, 1, 1, 1, 3, 1, 0, 0, 0, 0, 2, 3, 0, 1, 0, 0, 3, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 3, 1, 2, 0, 2, 1, 2, 2, 1, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 0, 0, 1, 0, 0]


In [74]:
l = []
for i in range(200):
    l.append(data1[i][3].index(1))
print(l)

[0, 2, 0, 1, 0, 2, 0, 0, 1, 0, 1, 2, 0, 2, 1, 1, 1, 0, 1, 2, 0, 0, 0, 0, 0, 1, 3, 0, 2, 0, 1, 2, 0, 1, 2, 0, 2, 1, 0, 2, 0, 0, 0, 2, 0, 1, 0, 2, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 2, 2, 0, 2, 2, 0, 0, 0, 0, 0, 1, 1, 2, 2, 2, 0, 2, 1, 0, 2, 1, 0, 3, 1, 1, 0, 2, 1, 2, 0, 2, 1, 1, 2, 2, 1, 2, 2, 0, 1, 1, 2, 2, 0, 0, 3, 1, 1, 0, 1, 1, 0, 1, 2, 0, 2, 1, 0, 1, 2, 1, 1, 1, 0, 0, 1, 1, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 1, 0, 0, 2, 0, 2, 1, 2, 2, 2, 1, 1, 2, 0, 0, 1, 1, 1, 1, 1, 1, 1, 3, 2, 1, 1, 0, 1, 2, 2, 0, 0, 3, 3, 0, 2, 0, 3, 0, 0, 0, 0, 0, 0, 1, 1, 0, 2, 1, 1, 1, 0, 0, 1, 2, 1, 0, 0, 2, 2, 0, 2, 0, 1]


In [75]:
y_test = lista
y_pred = l
matriz_conf = confusion_matrix(y_true = y_test,y_pred=y_pred)
print(matriz_conf)

[[41 31 27  5]
 [21 22 16  0]
 [10  9 10  1]
 [ 3  2  1  1]]


**Testing Guarnición**

In [76]:
lista = []
for item in guar:
    lista.append(list(item).index(max(item)))
print(lista)

[2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2, 1, 2, 0, 2, 2, 0, 2, 2, 0, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 0, 2, 0, 1, 2, 0, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 0, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2]


In [77]:
l = []
for i in range(200):
    l.append(data1[i][4].index(1))
print(l)

[2, 1, 2, 0, 0, 0, 2, 1, 2, 0, 2, 2, 2, 1, 0, 0, 2, 1, 0, 1, 0, 2, 0, 2, 0, 0, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 2, 0, 0, 2, 2, 1, 2, 2, 0, 2, 2, 2, 0, 2, 0, 0, 2, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0, 0, 2, 2, 0, 1, 0, 0, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 1, 2, 2, 0, 0, 2, 2, 2, 0, 1, 0, 2, 1, 2, 0, 2, 2, 2, 2, 2, 1, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 1, 2, 2, 2, 2, 0, 2, 0, 2, 2, 0, 0, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 0, 1, 2, 0, 0, 2, 2, 0, 2, 2, 2, 2, 0, 0, 1, 2, 2, 1, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 0, 2, 0]


In [78]:
y_test = lista
y_pred = l
matriz_conf = confusion_matrix(y_true = y_test,y_pred=y_pred)
print(matriz_conf)

[[12  1  9]
 [ 7  2  8]
 [48 14 99]]
